# Manipulating DataFrames with Pandas - 2 (Grouping Data)

We can use `groupby()` function to
* Split data into groups
* Apply a function to each group
  - Aggregate (e.g. compute counts, means)
  - Transform (e.g. fill NAs within groups with group specific values)
  - Filter (filter out data based on a group statistic)
* Combine the results

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

We'll use _Gapminder_ dataset in this Notebook.

In [2]:
df = pd.read_csv('data/gapminder.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10111 entries, 0 to 10110
Data columns (total 8 columns):
Country            10111 non-null object
Year               10111 non-null int64
fertility          10100 non-null float64
life               10111 non-null float64
population         10108 non-null float64
child_mortality    9210 non-null float64
gdp                9000 non-null float64
region             10111 non-null object
dtypes: float64(5), int64(1), object(2)
memory usage: 632.0+ KB


>Aggregating functions are the ones that reduce the dimension of the returned objects. Some common aggregating functions are tabulated below:
- mean()	Compute mean of groups
- sum()	Compute sum of group values
- size()	Compute group sizes
- count()	Compute count of group
- std()	Standard deviation of groups
- var()	Compute variance of groups
- sem()	Standard error of the mean of groups
- describe()	Generates descriptive statistics
- first()	Compute first of group values
- last()	Compute last of group values
- nth()	Take nth value, or a subset if n is a list
- min()	Compute min of group values
- max()	Compute max of group values.
[source](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)

In [3]:
df.groupby('region').mean()

,Year,fertility,life,population,child_mortality,gdp
region,,,,,,
America,1988.500000,3.486061,68.722251,1.774572e+07,50.513292,11599.921875
East Asia & Pacific,1988.510931,3.725836,66.108632,5.468619e+07,59.337826,13336.156923
Europe & Central Asia,1988.550781,2.214177,71.931303,1.600358e+07,30.180168,18442.045417
Middle East & North Africa,1988.500000,4.970019,65.194301,1.171303e+07,69.884533,27510.731579
South Asia,1988.500000,5.004162,57.137710,1.406782e+08,137.767150,2552.650000
Sub-Saharan Africa,1988.500000,5.956105,51.664426,1.050998e+07,158.917473,3152.428511


In [4]:
df.groupby(['Year', 'region']).mean().head()

fertility       life    population  \
Year region                                                           
1964 America                      5.574650  60.462775  1.155489e+07   
     East Asia & Pacific          5.708032  56.798429  3.470839e+07   
     Europe & Central Asia        3.270488  67.840110  1.370303e+07   
     Middle East & North Africa   6.965571  52.119810  5.623595e+06   
     South Asia                   6.480500  43.877125  7.813424e+07   

                                 child_mortality           gdp  
Year region                                                     
1964 America                          113.950667   6813.875000  
     East Asia & Pacific              129.109130   6431.500000  
     Europe & Central Asia             61.585319   9760.062500  
     Middle East & North Africa       179.605263  10962.157895  
     South Asia                       256.922500   1233.875000

Multiple aggregations can be applied with `.agg()` method.

In [5]:
df.groupby('region')['population'].agg(['max', 'mean'])

,max,mean
region,,
America,3.184976e+08,1.774572e+07
East Asia & Pacific,1.359368e+09,5.468619e+07
Europe & Central Asia,1.489456e+08,1.600358e+07
Middle East & North Africa,8.537844e+07,1.171303e+07
South Asia,1.275138e+09,1.406782e+08
Sub-Saharan Africa,1.709011e+08,1.050998e+07


In [7]:
df = pd.read_csv('data/gapminder.csv', index_col = ['region', 'Year'])
df.head()

Country  fertility    life  population  child_mortality  \
region     Year                                                                
South Asia 1964  Afghanistan      7.671  33.639  10474903.0            339.7   
           1965  Afghanistan      7.671  34.152  10697983.0            334.1   
           1966  Afghanistan      7.671  34.662  10927724.0            328.7   
           1967  Afghanistan      7.671  35.170  11163656.0            323.3   
           1968  Afghanistan      7.671  35.674  11411022.0            318.1   

                    gdp  
region     Year          
South Asia 1964  1182.0  
           1965  1182.0  
           1966  1168.0  
           1967  1173.0  
           1968  1187.0